In [1]:
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
import joblib
from collections import Counter
height, width = 64, 64

In [2]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.25)
label_counts = Counter(data.iloc[:, 0])
print(label_counts)

Counter({0: 1215, 50: 152, 10: 141, 76: 141, 69: 139, 29: 136, 3: 134, 64: 134, 67: 133, 80: 133, 93: 133, 61: 132, 32: 132, 13: 131, 38: 129, 44: 129, 48: 129, 78: 129, 47: 129, 90: 129, 37: 128, 87: 128, 19: 126, 39: 126, 25: 126, 52: 126, 56: 126, 75: 126, 20: 125, 4: 125, 83: 124, 33: 123, 11: 123, 21: 123, 86: 122, 42: 121, 35: 121, 30: 121, 57: 120, 36: 120, 17: 120, 84: 119, 27: 119, 88: 119, 73: 119, 89: 119, 54: 118, 40: 118, 7: 118, 85: 118, 60: 117, 23: 117, 2: 117, 22: 116, 91: 116, 63: 116, 16: 116, 6: 116, 45: 115, 41: 115, 15: 115, 51: 114, 72: 114, 28: 114, 49: 114, 18: 114, 34: 114, 71: 114, 62: 113, 55: 113, 5: 112, 12: 112, 46: 112, 74: 112, 94: 111, 77: 111, 1: 111, 53: 111, 95: 111, 82: 110, 58: 110, 81: 110, 68: 109, 31: 109, 59: 108, 26: 108, 79: 106, 70: 105, 9: 105, 92: 104, 8: 104, 24: 99, 66: 98, 43: 98, 65: 96, 14: 91})


In [3]:
X = data.iloc[:, 1:].astype("float64")
y = data.iloc[:, 0].astype("float64")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, f'../artifacts/random_forest_model_{height}_x_{width}.pkl')

['../artifacts/random_forest_model_64_x_64.pkl']

In [5]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (2500, 4096)
Train Accuracy: 99.6800%
Test Accuracy: 51.4000%
F1 Score: 51.8168%
Recall: 51.4000%


In [6]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(np.array(X))
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

In [7]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, f'../artifacts/random_forest_model_hog_{height}_x_{width}.pkl')

['../artifacts/random_forest_model_hog_64_x_64.pkl']

In [8]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (2500, 9216)
Train Accuracy: 99.6800%
Test Accuracy: 57.2000%
F1 Score: 57.0319%
Recall: 57.2000%


In [9]:
import cv2

def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        # If no keypoints are found, use a zero array of the same length as a typical descriptor
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        # Flatten descriptors and use them as features
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

# Extract SIFT features
X_sift = extract_sift_features(np.array(X))

In [10]:
X_sift = extract_sift_features(np.array(X))

# Since the number of features might vary, we need to ensure consistent feature vector size
# Here, we'll pad with zeros to the maximum descriptor length found
max_len = max(len(f) for f in X_sift)
X_sift = np.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)

In [11]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, f'../artifacts/random_forest_model_sift_{height}_x_{width}.pkl')

['../artifacts/random_forest_model_sift_64_x_64.pkl']

In [12]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (2500, 128)
Train Accuracy: 9.3800%
Test Accuracy: 11.0800%
F1 Score: 2.2104%
Recall: 11.0800%
